In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf

cocomo=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\cocomo81_dataset.csv",header=None)
columns_cocomo=['rely','data','cplx','time','stor','virt','turn','acap','aexp','pcap','vexp','lexp','modp','tool','sced','loc','actual']
cocomo.set_axis(columns_cocomo,axis='columns',inplace=True)
cocomo.set_axis(range(1,64),axis=0 ,inplace=True)
cocomo.rename_axis("Features", axis=1,inplace=True)
cocomo.rename_axis("Projects", axis=0,inplace=True)

In [2]:


# Step 1: Collect Data (Replace with your actual data)
# Load your dataset into 'X' (features) and 'y' (effort/target variable)
# Feature matrix X should have shape (n_samples, n_features)
X = cocomo.drop('actual',axis=1).values
# Target values y should have shape (n_samples,)
y = cocomo['actual'].values

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)


# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    cross_val_rmse = []
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=0)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
        
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)
#====================================================================================================================

pbounds_HS = {
    'hms' : (10,30),
    'iterations': (30,100),
    'harmony_rate' : (0.1,0.9)
}

# Harmony Search Parameters
def harmony_search(hms,iterations, harmony_rate):
    num_features = X.shape[1]
    hms = int(hms)  # Harmony memory size
    iterations = int(iterations)  # Number of iterations

    # Initialize harmony memory
    harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

    # Harmony Search Algorithm
    for _ in range(iterations):
        rnd_choice=np.random.choice(hms)
        new_harmony = np.copy(harmony_memory[rnd_choice])
        for i in range(num_features):
            if np.random.rand() < harmony_rate:  # Adjust this probability based on your problem
                new_harmony[i] = 1 - new_harmony[i]
        current_obj = objective_function(harmony_memory[rnd_choice])
        new_obj = objective_function(new_harmony)
        if new_obj > current_obj:
            harmony_memory[rnd_choice] = new_harmony
    return max([objective_function(i) for i in harmony_memory])

optimizer_HS = BayesianOptimization(
        f=harmony_search,
        pbounds=pbounds_HS,
        random_state=42,
        verbose=1,
    )

optimizer_HS.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.
# Print the best hyperparameters found.
best_params_HS = optimizer_HS.max['params']

num_features = X.shape[1]
hms = int(best_params_HS['hms'])  # Harmony memory size
iterations = int(best_params_HS['iterations'])  # Number of iterations
harmony_rate= best_params_HS['harmony_rate']

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < harmony_rate:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj > current_obj:
        harmony_memory[rnd_choice] = new_harmony
    
#======================================================================================================================

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print('Feature selected')
if len(selected_features) == 0:
    return float('-inf')  # Penalize subsets with no selected features
    
    
# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
        'neurons_input': (10, 50),
        'neurons_hidden': (10, 50),
        'num_hidden_layers': (1, 5),
        'learning_rate': (1e-5, 1e-2),
        'batch_size': (8,32 ),
        'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)
    
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

|   iter    |  target   | harmon... |    hms    | iterat... |
-------------------------------------------------------------
1/1 [==============================] - 0s 40ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 36ms/step


KeyboardInterrupt: 

In [7]:


# Step 1: Collect Data (Replace with your actual data)
# Load your dataset into 'X' (features) and 'y' (effort/target variable)
# Feature matrix X should have shape (n_samples, n_features)
X = cocomo.drop('actual',axis=1).values
# Target values y should have shape (n_samples,)
y = cocomo['actual'].values

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)


# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    cross_val_rmse = []
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=0)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
        
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)
#====================================================================================================================
# Harmony Search Parameters
num_features = X.shape[1]
hms = 20  # Harmony memory size
iterations = 100  # Number of iterations

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < 0.5:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj < current_obj:
        harmony_memory[rnd_choice] = new_harmony

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
#======================================================================================================================

#if len(selected_features) == 0:
#   return float('-inf')  # Penalize subsets with no selected features
    
    
# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
        'neurons_input': (10, 50),
        'neurons_hidden': (10, 50),
        'num_hidden_layers': (1, 5),
        'learning_rate': (1e-5, 1e-2),
        'batch_size': (8,32 ),
        'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)
    
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 34ms/step


...

1/1 [==============================] - 0s 35ms/step
| 38        | -432.4    | 11.67     | 18.77     | 0.00715   | 31.15     | 23.38     | 1.605     |
1/1 [==============================] - 0s 44ms/step
| 39        | -480.1    | 11.22     | 19.94     | 0.006574  | 36.67     | 24.13     | 3.785     |
1/1 [==============================] - 0s 36ms/step
| 40        | -620.4    | 13.29     | 17.68     | 0.003131  | 33.35     | 19.53     | 1.845     |
1/1 [==============================] - 0s 35ms/step
|   iter    |  target   | batch_... |  epochs   | learni... | neuron... | neuron... | num_hi... |
-------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 36ms/step
| 1         | -3.345e+0 | 16.99     | 19.26     | 0.007323  | 33.95     | 16.24     | 1.624     |
1/1 [==============================] - 0s 47ms/step
| 2         | -3.066e+0 | 9.394     | 17.99     | 0.006015  | 38.32     | 10.82     | 4.88      |
1/

1/1 [==============================] - 0s 41ms/step
| 35        | -3.63e+03 | 20.82     | 12.09     | 0.0002202 | 33.14     | 35.18     | 3.528     |
1/1 [==============================] - 0s 42ms/step
| 36        | -3.078e+0 | 10.02     | 16.6      | 0.005636  | 38.82     | 10.24     | 3.835     |
1/1 [==============================] - 0s 41ms/step
| 37        | -3.614e+0 | 27.8      | 6.016     | 0.009573  | 23.59     | 11.92     | 3.756     |
1/1 [==============================] - 0s 47ms/step
| 38        | -3.513e+0 | 21.02     | 7.68      | 0.004659  | 42.14     | 47.59     | 4.96      |
1/1 [==============================] - 0s 43ms/step
| 39        | -3.115e+0 | 8.429     | 16.16     | 0.004637  | 40.03     | 11.41     | 3.535     |
1/1 [==============================] - 0s 46ms/step
| 40        | -3.037e+0 | 8.193     | 18.61     | 0.004628  | 37.27     | 10.8      | 4.831     |
1/1 [==============================] - 0s 56ms/step
|   iter    |  target   | batch_... |  epochs   

1/1 [==============================] - 0s 41ms/step
| 32        | -2.538e+0 | 14.85     | 13.56     | 0.007422  | 46.53     | 13.17     | 3.828     |
1/1 [==============================] - 0s 38ms/step
| 33        | -279.5    | 20.87     | 19.84     | 0.006639  | 10.14     | 30.0      | 2.181     |
1/1 [==============================] - 0s 44ms/step
| 34        | -1.249e+0 | 9.572     | 9.772     | 0.004336  | 49.95     | 12.57     | 3.785     |
1/1 [==============================] - 0s 42ms/step
| 35        | -424.8    | 20.82     | 12.09     | 0.0002202 | 33.14     | 35.18     | 3.528     |
1/1 [==============================] - 0s 35ms/step
| 36        | -407.1    | 21.55     | 13.64     | 0.007056  | 13.22     | 15.76     | 1.572     |
1/1 [==============================] - 0s 41ms/step
| 37        | -410.1    | 27.8      | 6.016     | 0.009573  | 23.59     | 11.92     | 3.756     |
1/1 [==============================] - 0s 35ms/step
| 38        | -425.5    | 22.29     | 5.297     

1/1 [==============================] - 0s 38ms/step
| 29        | -646.9    | 29.95     | 6.991     | 0.004999  | 41.0      | 30.14     | 1.329     |
1/1 [==============================] - 0s 71ms/step
| 30        | -644.1    | 23.13     | 8.055     | 0.003133  | 17.94     | 16.38     | 4.888     |
1/1 [==============================] - 0s 37ms/step
| 31        | -647.6    | 27.88     | 15.54     | 0.001176  | 26.81     | 12.74     | 1.15      |
1/1 [==============================] - 0s 34ms/step
| 32        | -603.2    | 25.61     | 16.21     | 0.00591   | 25.08     | 32.92     | 1.83      |
1/1 [==============================] - 0s 37ms/step
| 33        | -475.1    | 18.29     | 19.3      | 0.004937  | 32.58     | 18.48     | 2.392     |
1/1 [==============================] - 0s 42ms/step
| 34        | -476.4    | 27.24     | 10.52     | 0.01      | 30.18     | 27.15     | 3.1       |
1/1 [==============================] - 0s 38ms/step
| 35        | -598.5    | 26.47     | 11.56     

1/1 [==============================] - 0s 43ms/step
| 26        | -1.424e+0 | 11.63     | 17.74     | 0.01      | 40.86     | 16.13     | 3.439     |
1/1 [==============================] - 0s 36ms/step
| 27        | -1.802e+0 | 25.9      | 20.0      | 0.01      | 22.91     | 30.57     | 1.0       |
1/1 [==============================] - 0s 35ms/step
| 28        | -2.063e+0 | 8.095     | 18.45     | 1e-05     | 41.27     | 16.13     | 1.0       |
1/1 [==============================] - 0s 51ms/step
| 29        | -1.301e+0 | 10.39     | 16.04     | 0.01      | 39.41     | 17.67     | 5.0       |
1/1 [==============================] - 0s 52ms/step
| 30        | -1.458e+0 | 11.39     | 18.31     | 0.01      | 40.01     | 19.84     | 5.0       |
1/1 [==============================] - 0s 49ms/step
| 31        | -1.334e+0 | 12.09     | 16.47     | 0.01      | 38.36     | 14.18     | 5.0       |
1/1 [==============================] - 0s 50ms/step
| 32        | -1.525e+0 | 13.48     | 13.97     